In [1]:
import numpy as np
import pandas as pd
import random, os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# import tensorflow as tf
# from tensorflow import lite
# from tensorflow import keras
# from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
# from matplotlib.image import imread
# from tensorflow.keras.metrics import categorical_accuracy, AUC
# from sklearn.model_selection import train_test_split

In [2]:
# Add an additional column, mapping to the type
Image_info_df = pd.read_csv('../Data/Gaussian_Retina_Data/train.csv')

diagnosis_dict_binary = {
    0: 'No_DR',
    1: 'DR',
    2: 'DR',
    3: 'DR',
    4: 'DR'
}

diagnosis_dict = {
    0: 'No_DR',
    1: 'Mild',
    2: 'Moderate',
    3: 'Severe',
    4: 'Proliferate_DR',
}


Image_info_df['binary_type'] =  Image_info_df['diagnosis'].map(diagnosis_dict_binary.get)
Image_info_df['type'] = Image_info_df['diagnosis'].map(diagnosis_dict.get)
# Image_info_df.head()

In [13]:
special = [3453] 
# special = [190,279,296,312,465,492,619,671,700,1046,1097,1225,1289,1619,2188,2255,2521,2708,2713,2818,3063,3228,3291,3296,3453]

special_df=Image_info_df.loc[special]

In [14]:
print(special_df['type'].value_counts(), '\n')

Proliferate_DR    1
Name: type, dtype: int64 



In [15]:
special_df

,id_code,diagnosis,binary_type,type
3453,f0098e9d4aee,4,DR,Proliferate_DR


In [16]:
# Hack to provide labels
# train_path = 'train'
# train_batches = ImageDataGenerator(rescale = 1./255).flow_from_directory(train_path, target_size=(224,224), shuffle = True)

In [17]:
# Create working directories for train/val/test
base_dir = ''

special_dir = os.path.join(base_dir, 'Special')

if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

if os.path.exists(special_dir):
    shutil.rmtree(special_dir)
os.makedirs(special_dir)

In [18]:
# # Copy images to respective working directory
src_dir = '../Data/Gaussian_Retina_Data/gaussian_filtered_images/gaussian_filtered_images/'
for index, row in special_df.iterrows():
    diagnosis = row['type']
    binary_diagnosis = row['binary_type']
    id_code = row['id_code'] + ".png"
    srcfile = os.path.join(src_dir, diagnosis, id_code)
    dstfile = os.path.join(special_dir, diagnosis)
    os.makedirs(dstfile, exist_ok = True)
    shutil.copy(srcfile, dstfile)

In [19]:
# Setting up ImageDataGenerator for train/val/test 

special_path = 'Special'

Diagnosis = ImageDataGenerator(rescale = 1./255).flow_from_directory(special_path, target_size=(224,224), shuffle = True)

Found 1 images belonging to 1 classes.


In [20]:
# Load the model
from tensorflow.keras.models import load_model
retinopathy_model = load_model("Models/Retinopathy_model_trained_20-40-40.h5")

In [21]:
predicted = retinopathy_model.predict(Diagnosis)
predicted = np.argmax(predicted,axis=1)

# Map the label
# labels = (Diagnosis.class_indices)
# labels = dict((v,k) for k,v in labels.items())
Labels = {0: 'Mild', 1: 'Moderate', 2: 'No_DR', 3: 'Proliferate_DR', 4: 'Severe'}
predicted = [Labels[k] for k in predicted]

# Display the result
print(f'The prediction is: {predicted[:1]}')

The prediction is: ['Proliferate_DR']


In [22]:
predicted[:1]

['Proliferate_DR']